In [ ]:
AI Advatars

Use AI to evaluate an advertising creative against the most likely buyer personas. This colab walks you out through 
the step by step process so we can get ideas, customize them, and build your own client specific avatars

1. Configure Gemini AI to drive the avatars.
2. Star with a product description.
3. Gemini will find the best avatars to evaluate
4. Reliably move the avatars into Python structures
5. Use each avatar to:

a. Evaluate an existing image creative to get edit suggestions.
b. Prioritize personas to invest in custom campaigns
c. Get user journey insights into the persona
d. Create an audience targeting plan for each persona.
e. Generate keyword lists
f. Create an effective landing page template for each avatar.

1. Create summary sheet of each avatar for the campaign

This is advertising specific toolkit to get started with AI and advertising creatives. Key concepts you can leverage 
in your day to day are:

a. Examples of advertising prompts.
b. Practical means of moving data from AI to Python for processing.
c. Using Gemini for text and fir images in tandem.
d. Building a data structure that can be fed into a Google Ads Product.

Every step in this workbook builds on the last, so if we modify a step, remember to re-run the next steps in
the sequence. To help to tune the prompts each step is designed to be run as many times that need to get the 
output to where like it. Although can use this workbook without editing code, the prompts are intended to be customized 
and evolved to the product and needs 

Prompts to adjust fields and content.
Data Structures to add or remove avatars for example.
Models the Gemini Text model is better at text, use Gemini Vision Only for images.
Sections to add entirely new data for each avatar. 

Setting Expectations

1. This is an AI integration, things may format incorrectly, try re-running a step if it errors on parsing.
2. To see output may need to expand the cells.

Setup

Install Vertex AI SDK


In [ ]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

In [ ]:
#Automatically restart kernel after installs so that your environment can access the new packages

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Authenticating the notebook environment

If using Colab to run this notebook, uncomment the cell below and continue. If using Vertex AI Workbench, check out the setup instructions here (https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)

In [ ]:
from google.colab import auth

auth.authenticate_user()

Set the project ID and initialize Vertex AI SDK

Get PROJECT ID using gcloud

In [ ]:
#Define project informarion
PROJECT_ID = "gtech-kenjora" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

Import Libraries

In [ ]:
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part

AI Advatars

In [ ]:
#@title Add Product Description To Get Avatars
#@markdown This product description will be used to build the information

product = "mazda 3 hatchback" # @param {type:"string"}

In [ ]:
#@title Add an existing Creative to evaluate against avatars
#@markdown This image will be evaluated by the avatars
#@markdown Click on the folder icon on the left and upload an image, then paste the path here

from base64 import b64encode
from IPython.display import HTML, Markdown
from PIL import Image as PIL_Image

def embed_image(path_to_image):
    try:
       img = open(path_to_image, 'rb').read()
       return 'data:image/jpeg;base64,' + b64encode(img).decode()
    except FileNotFoundError:
       return ''

picture_url = '/content/creative.jpg' # @param {type:"string"}

try:
   picture = PIL_Image.open(picture_url)
except OSError:
   picture = None
   print('WARNING: No uploaded creative detected, some parts of this workbook ay not work.')

HTML(f'')

In [ ]:
#@title Set Up Imports And Helpers Plus Initialize Chat
#@markdown This just sets up common 